In [141]:
import pandas as pd
import numpy as np

In [142]:
data = pd.read_csv('alz_data.csv')
data = data.drop(columns='Unnamed: 0')
data.head

<bound method NDFrame.head of     Age  GDS  CDR  Delayed Recall  Number Span Test Forward  \
0    65    1  0.0              28                        10   
1    65    1  0.0              30                        10   
2    65    1  0.0              35                        10   
3    65    1  0.0              15                        10   
4    65    1  0.0              26                        12   
..  ...  ...  ...             ...                       ...   
95   80    6  3.0               1                         2   
96   92    6  3.0               2                         2   
97   75    7  3.0               1                         1   
98   79    7  3.0               0                         0   
99   74    7  3.0               0                         0   

    Part A Trail Making  Part B Trail Making  moca  Categories Animal  
0                    25                   45    27                 25  
1                    24                   45    30                 26

In [143]:
zs = data
for row in range(zs.shape[0]):
    if zs.iloc[row, 1] < 60:
        zs = zs.assign(Delayed_Recall_Z = lambda x: ((x['Delayed Recall']-20.9)/7.0))
        zs = zs.assign(Number_Span_Test_Forward_Z = lambda x: ((x['Number Span Test Forward']-8.7)/2.4))
        zs = zs.assign(Part_A_Trail_Making_Z = lambda x: ((-x['Part A Trail Making']+22.3)/8.4))
        zs = zs.assign(Part_B_Trail_Making_Z = lambda x: ((-x['Part B Trail Making']+55.4)/27.2))
        zs = zs.assign(MoCA_Z = lambda x: ((x['moca']-27.5)/2.1))
        zs = zs.assign(Categories_Animal_Z = lambda x: ((x['Categories Animal']-23.6)/5.3))
    if 60 <= zs.iloc[row, 1] <= 69:
        zs = zs.assign(Delayed_Recall_Z = lambda x: ((x['Delayed Recall']-20.4)/6.4))
        zs = zs.assign(Number_Span_Test_Forward_Z = lambda x: ((x['Number Span Test Forward']-8.4)/2.3))
        zs = zs.assign(Part_A_Trail_Making_Z = lambda x: ((-x['Part A Trail Making']+27.3)/9.6))
        zs = zs.assign(Part_B_Trail_Making_Z = lambda x: ((-x['Part B Trail Making']+70.1)/33.8))
        zs = zs.assign(MoCA_Z = lambda x: ((x['moca']-26.9)/2.4))
        zs = zs.assign(Categories_Animal_Z = lambda x: ((x['Categories Animal']-22.7)/5.5))
    if 70 <= zs.iloc[row, 1] <= 79:
        zs = zs.assign(Delayed_Recall_Z = lambda x: ((x['Delayed Recall']-26.3)/2.7))
        zs = zs.assign(Number_Span_Test_Forward_Z = lambda x: ((x['Number Span Test Forward']-8.3)/2.3))
        zs = zs.assign(Part_A_Trail_Making_Z = lambda x: ((-x['Part A Trail Making']+31.0)/10.8))
        zs = zs.assign(Part_B_Trail_Making_Z = lambda x: ((-x['Part B Trail Making']+82.7)/41.4))
        zs = zs.assign(MoCA_Z = lambda x: ((x['moca']-26.3)/2.7))
        zs = zs.assign(Categories_Animal_Z = lambda x: ((x['Categories Animal']-21.2)/5.4))
    if 80 <= zs.iloc[row, 1] <= 89:
        zs = zs.assign(Delayed_Recall_Z = lambda x: ((x['Delayed Recall']-16.9)/6.6))
        zs = zs.assign(Number_Span_Test_Forward_Z = lambda x: ((x['Number Span Test Forward']-8.0)/2.3))
        zs = zs.assign(Part_A_Trail_Making_Z = lambda x: ((-x['Part A Trail Making']+36.2)/13.3))
        zs = zs.assign(Part_B_Trail_Making_Z = lambda x: ((-x['Part B Trail Making']+102.0)/55.2))
        zs = zs.assign(MoCA_Z = lambda x: ((x['moca']-25.3)/3.0))
        zs = zs.assign(Categories_Animal_Z = lambda x: ((x['Categories Animal']-19.5)/5.6))
    if 90 <= zs.iloc[row, 1]:
        zs = zs.assign(Delayed_Recall_Z = lambda x: ((x['Delayed Recall']-15.1)/5.8))
        zs = zs.assign(Number_Span_Test_Forward_Z = lambda x: ((x['Number Span Test Forward']-7.6)/2.1))
        zs = zs.assign(Part_A_Trail_Making_Z = lambda x: ((-x['Part A Trail Making']+42.0)/13.4))
        zs = zs.assign(Part_B_Trail_Making_Z = lambda x: ((-x['Part B Trail Making']+140.6)/75.3))
        zs = zs.assign(MoCA_Z = lambda x: ((x['moca']-23.8)/3.5))
        zs = zs.assign(Categories_Animal_Z = lambda x: ((x['Categories Animal']-17.0)/5.4))

zs = zs.drop(['Age', 'GDS', 'CDR','Delayed Recall','Number Span Test Forward','Part A Trail Making','Part B Trail Making','moca','Categories Animal'], axis = 1)

In [144]:
print(zs)

    Delayed_Recall_Z  Number_Span_Test_Forward_Z  Part_A_Trail_Making_Z  \
0           1.014286                    0.541667              -0.321429   
1           1.300000                    0.541667              -0.202381   
2           2.014286                    0.541667              -0.916667   
3          -0.842857                    0.541667              -0.916667   
4           0.728571                    1.375000              -0.202381   
..               ...                         ...                    ...   
95         -2.842857                   -2.791667             -15.202381   
96         -2.700000                   -2.791667             -15.202381   
97         -2.842857                   -3.208333             -15.202381   
98         -2.985714                   -3.625000             -15.202381   
99         -2.985714                   -3.625000             -15.202381   

    Part_B_Trail_Making_Z     MoCA_Z  Categories_Animal_Z  
0                0.382353  -0.238095   

In [145]:
zs_ab = zs
zs_ab['DR_ab'] = zs_ab['Delayed_Recall_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['NSF_ab'] = zs_ab['Number_Span_Test_Forward_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['TMA_ab'] = zs_ab['Part_A_Trail_Making_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['TMB_ab'] = zs_ab['Part_B_Trail_Making_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['MoCA_ab'] = zs_ab['MoCA_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab['Cat_An_ab'] = zs_ab['Categories_Animal_Z'].apply(lambda x: 1 if x <= -1.5 else 0)
zs_ab = zs_ab.drop(['Delayed_Recall_Z','Number_Span_Test_Forward_Z','Part_A_Trail_Making_Z','Part_B_Trail_Making_Z','MoCA_Z','Categories_Animal_Z'], axis = 1)


In [146]:
print(zs_ab)

    DR_ab  NSF_ab  TMA_ab  TMB_ab  MoCA_ab  Cat_An_ab
0       0       0       0       0        0          0
1       0       0       0       0        0          0
2       0       0       0       0        0          0
3       0       0       0       1        0          0
4       0       0       0       0        0          0
..    ...     ...     ...     ...      ...        ...
95      1       1       1       1        1          1
96      1       1       1       1        1          1
97      1       1       1       1        1          1
98      1       1       1       1        1          1
99      1       1       1       1        1          1

[100 rows x 6 columns]


In [147]:
data = data.drop(['Age', 'Delayed Recall','Number Span Test Forward','Part A Trail Making','Part B Trail Making','moca','Categories Animal'], axis = 1)
print(data)
data = data.join(zs_ab)
print(data)

    GDS  CDR
0     1  0.0
1     1  0.0
2     1  0.0
3     1  0.0
4     1  0.0
..  ...  ...
95    6  3.0
96    6  3.0
97    7  3.0
98    7  3.0
99    7  3.0

[100 rows x 2 columns]
    GDS  CDR  DR_ab  NSF_ab  TMA_ab  TMB_ab  MoCA_ab  Cat_An_ab
0     1  0.0      0       0       0       0        0          0
1     1  0.0      0       0       0       0        0          0
2     1  0.0      0       0       0       0        0          0
3     1  0.0      0       0       0       1        0          0
4     1  0.0      0       0       0       0        0          0
..  ...  ...    ...     ...     ...     ...      ...        ...
95    6  3.0      1       1       1       1        1          1
96    6  3.0      1       1       1       1        1          1
97    7  3.0      1       1       1       1        1          1
98    7  3.0      1       1       1       1        1          1
99    7  3.0      1       1       1       1        1          1

[100 rows x 8 columns]


In [148]:
def alz_diagnose(data):
    diags = []
    for row in range(data.shape[0]):
        row_sum = data.iloc[row].sum()
        try:
            if 1.0 <= row_sum <= 5.0:
                if data.iloc[row, 0] == 1:
                    diags.append('Normal')
                if data.iloc[row, 0] == 2:
                    diags.append('SCD')
            elif 6.5 <= row_sum <= 7.5:
                diags.append('MCI')
            elif 8 <= row_sum <= 10:
                diags.append('Probable Dementia')
            elif 11 <= row_sum <= 12:
                diags.append('Mild Dementia')
            elif 13 <= row_sum <= 15:
                diags.append('Moderate Dementia')
            elif row_sum == 16:
                diags.append('Severe Dementia')
            else:
                raise ValueError()
        except ValueError:
            diags.append('This combination of diagnostic values does not indicate clear diagnosis. Please verify values are correct.')
            pass
    return diags

In [149]:
data_diag = alz_diagnose(data)
print(data_diag)

['Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'Normal', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'SCD', 'MCI', 'MCI', 'MCI', 'Probable Dementia', 'MCI', 'MCI', 'This combination of diagnostic values does not indicate clear diagnosis. Please verify values are correct.', 'MCI', 'MCI', 'Probable Dementia', 'MCI', 'MCI', 'Probable Dementia', 'MCI', 'MCI', 'Probable Dementia', 'Probable Dementia', 'Probable Dementia', 'Probable Dementia', 'Mild Dementia', 'Mild Dementia', 'Mild Dementia', 'Mild Dementia', 'Probable Dementia', 'Mild Dementia', 'Mild Dementia', 'Mild Dementia', 'Mild Dementia', 'Mild Dementia', 'Mild Dementia', 'Mild Dementia', 'Mi

In [150]:
df5 = pd.DataFrame({'Diagnosis':data_diag})
data = data.join(df5)
print(data)


    GDS  CDR  DR_ab  NSF_ab  TMA_ab  TMB_ab  MoCA_ab  Cat_An_ab  \
0     1  0.0      0       0       0       0        0          0   
1     1  0.0      0       0       0       0        0          0   
2     1  0.0      0       0       0       0        0          0   
3     1  0.0      0       0       0       1        0          0   
4     1  0.0      0       0       0       0        0          0   
..  ...  ...    ...     ...     ...     ...      ...        ...   
95    6  3.0      1       1       1       1        1          1   
96    6  3.0      1       1       1       1        1          1   
97    7  3.0      1       1       1       1        1          1   
98    7  3.0      1       1       1       1        1          1   
99    7  3.0      1       1       1       1        1          1   

            Diagnosis  
0              Normal  
1              Normal  
2              Normal  
3              Normal  
4              Normal  
..                ...  
95  Moderate Dementia  
96 